In [13]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Load and format data

In [22]:
# Load VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Load the JSON file into a DataFrame
file_path = 'data/sarcasm_data.json'
df = pd.read_json(file_path).transpose()

# Reset the index to turn the first element into a new column
df = df.reset_index()

# Define a function to apply sentiment analysis to a text
def get_sentiment(text):
    return analyzer.polarity_scores(text)

# Apply sentiment analysis to the 'utterance' column
df['sentiment_utterance'] = df['utterance'].apply(get_sentiment)

# Apply sentiment analysis to the 'context' column
df['sentiment_context_all'] = df['context'].apply(get_sentiment)

# Apply sentiment analysis to each sentence in the 'context' column
df['sentiment_context_per_sentence'] = df['context'].apply(lambda context: [get_sentiment(sentence) for sentence in context])

# Visualize the DataFrame
df

if False:
    # Transform the 'sentiment_utterance' column into arrays of scores
    df['sentiment_utterance'] = df['sentiment_utterance'].apply(lambda x: list(x.values()))

    # add the context scores to the sentiment_utterance column
    df['sentiment_utterance'] = df['sentiment_utterance'] + df['sentiment_context_all'].apply(lambda x: list(x.values()))

    # Transpose the dataframe to get the orignal json format
    df = df.transpose()

    # Save the DataFrame to a new JSON file
    df.to_json('data/sarcasm_data_sentiment.json')

### Visualisation format SA context and utterance

In [4]:
df_context = df['sentiment_context_all']

print("context all sequences:", df['sentiment_context_all'][0])
df['sentiment_context_all'][0]
print("context per sequence:", df['sentiment_context_per_sentence'][0])
print("utterance:", df['sentiment_utterance'][0] )

context all sequences: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
context per sequence: [{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
utterance: {'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'compound': 0.3612}


# Format Data for Training

## Data with overal context sentiment

In [5]:
# Converts sarcasm True to 1 and False to 0
y = df['sarcasm'].astype(int).tolist()

context_all = df["sentiment_context_all"]
context_sentences = df["sentiment_context_per_sentence"]
utterance = df["sentiment_utterance"]

# Extracting 'neg', 'neu', and 'pos' per sentence and add index to keep context 
X = np.array([ [context_all[i]['neg'], context_all[i]['neu'], context_all[i]['pos'], utterance[i]['neg'], utterance[i]['neu'], utterance[i]['pos']] for i in range(context_all.shape[0]) ])

X_df = pd.DataFrame(X, columns=['context_neg', 'context_neu', 'context_pos', 'utterance_neg', 'utterance_neu', 'utterance_pos'])

# Split the dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_df

,context_neg,context_neu,context_pos,utterance_neg,utterance_neu,utterance_pos
0,0.000,1.000,0.000,0.000,0.783,0.217
1,0.000,0.871,0.129,0.180,0.820,0.000
2,0.143,0.857,0.000,0.000,1.000,0.000
3,0.000,0.906,0.094,0.058,0.851,0.091
4,0.097,0.815,0.088,0.000,1.000,0.000
...,...,...,...,...,...,...
685,0.000,1.000,0.000,0.102,0.898,0.000
686,0.062,0.751,0.187,0.000,0.858,0.142
687,0.506,0.494,0.000,0.000,0.763,0.237
688,0.000,1.000,0.000,0.000,0.781,0.219


## Data Sentence specific context sentiment

In [17]:
# Extracted keys for the floats
keys = ['neg', 'neu', 'pos']

# Initialize a list to hold the resulting arrays
X_sentences = []

# Update sarcasm labels
y_sentences = []

# Process each array (list of dictionaries)
for idx, sentences in enumerate(context_sentences):
    for entry in sentences:
        # Extract values for each key and concatenate with the index
        values = [entry.get(key, 0) for key in keys]

        # Create entries like so: index, negative score, neutral score, positive score 
        X_sentences.append([idx] + values + [utterance[idx]['neg'] , utterance[idx]['neu'] , utterance[idx]['pos']])

        # Update labels
        y_sentences.append(y[idx])
    
print("The size of the y and x match:",len(X_sentences)==len(y_sentences))

# Split the dataset into 80% training and 20% testing
Xs_train, Xs_test, ys_train, ys_test = train_test_split(X_sentences, y_sentences, test_size=0.2, random_state=42)

The size of the y and x match: True


In [18]:
pd.DataFrame(X_sentences, columns=['utterance idx', 'sentence_neg', 'sentence_neu' ,'sentence_pos', 'utterance_neg', 'utterance_neu', 'utterance_pos'])

,utterance idx,sentence_neg,sentence_neu,sentence_pos,utterance_neg,utterance_neu,utterance_pos
0,0,0.000,1.000,0.000,0.00,0.783,0.217
1,0,0.000,1.000,0.000,0.00,0.783,0.217
2,1,0.000,0.705,0.295,0.18,0.820,0.000
3,1,0.000,0.303,0.697,0.18,0.820,0.000
4,1,0.000,0.732,0.268,0.18,0.820,0.000
...,...,...,...,...,...,...,...
2256,688,0.000,1.000,0.000,0.00,0.781,0.219
2257,688,0.000,1.000,0.000,0.00,0.781,0.219
2258,688,0.000,0.000,1.000,0.00,0.781,0.219
2259,688,0.000,1.000,0.000,0.00,0.781,0.219


# Training Classifier

## Overal context score

In [19]:
# Create a Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)

Accuracy: 0.49
Confusion Matrix:
[[42 11]
 [60 25]]
Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.79      0.54        53
           1       0.69      0.29      0.41        85

    accuracy                           0.49       138
   macro avg       0.55      0.54      0.48       138
weighted avg       0.59      0.49      0.46       138



## Sentence specific context score

In [20]:
# Create a Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(Xs_train, ys_train)

# Predict on the test data
ys_pred = model.predict(Xs_test)

# Evaluate the model
accuracy = accuracy_score(ys_test, ys_pred)
conf_matrix = confusion_matrix(ys_test, ys_pred)
report = classification_report(ys_test, ys_pred)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)

Accuracy: 0.53
Confusion Matrix:
[[106 161]
 [ 54 132]]
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.40      0.50       267
           1       0.45      0.71      0.55       186

    accuracy                           0.53       453
   macro avg       0.56      0.55      0.52       453
weighted avg       0.58      0.53      0.52       453

